# Pediatric DS AML vs TAM

Sleuth Analysis with Kallisto Quantitation Input

## Following the Pachterlab walkthrough
[Using p-value aggregation to obtain gene differential expression in datasets with multiple experimental conditions](
https://pachterlab.github.io/sleuth_walkthroughs/pval_agg/analysis.html)




### Getting started with Sleuth

**sleuth** is a tool for the analysis and comparison of multiple related RNA-Seq experiments. Key features include:

* The ability to perform both transcript-level and gene-level analysis.
* Compatibility with kallisto enabling a fast and accurate workflow from reads to results.
* The use of boostraps to ascertain and correct for technical variation in experiments.
* An interactive app for exploratory data analysis.

To use sleuth, RNA-Seq data must first be quantified with kallisto ( which we did so with the CAVATICA workflow), which is a program for very fast RNA-Seq quantification based on pseudo-alignment. An important feature of kallisto is that it outputs bootstraps along with the estimates of transcript abundances. These can serve as proxies for technical replicates, allowing for an ascertainment of the variability in estimates due to the random processes underlying RNA-Seq as well as the statistical procedure of read assignment. kallisto can quantify 30 million human reads in less than 3 minutes on a Mac desktop computer using only the read sequences and a transcriptome index that itself takes less than 10 minutes to build. sleuth has been designed to work seamlessly and efficiently with kallisto, and therefore RNA-Seq analysis with kallisto and sleuth is tractable on a laptop computer in a matter of minutes. More details about kallisto and sleuth are provided the papers describing the methods:

#### Citations

* Nicolas L Bray, Harold Pimentel, Páll Melsted and Lior Pachter, Near-optimal probabilistic RNA-seq quantification, Nature Biotechnology 34, 525–527 (2016), doi:10.1038/nbt.3519

* Harold Pimentel, Nicolas L Bray, Suzette Puente, Páll Melsted and Lior Pachter, Differential analysis of RNA-seq incorporating quantification uncertainty, in press.

sleuth has been designed to facilitate the exploration of RNA-Seq data by utilizing the Shiny web application framework by RStudio. The worked example below illustrates how to load data into sleuth and how to open Shiny plots for exploratory data analysis. The code underlying all plots is available via the Shiny interface so that analyses can be fully “open source”.



### Introduction

Applying the same techniques used in the walk through - apply it to our own AML vs TAM

We need to set up our working directory.
And we need 

* `cowplot` - for making prettier plots and plots with grids.  and 
* `biomaRt` - for extracting the Ensembl transcript to gene mapping



In [1]:
install.packages("cowplot")


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
library("cowplot")


In [4]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.14")


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.18), R 4.1.1 (2021-08-10)

Installing package(s) 'BiocVersion'

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'backports', 'BiocGenerics', 'blob', 'brew', 'brio', 'broom',
  'bslib', 'callr', 'caret', 'class', 'cli', 'clipr', 'colorspace',
  'commonmark', 'conflicted', 'covr', 'cpp11', 'crayon', 'credentials',
  'crosstalk', 'curl', 'data.table', 'DBI', 'dbplyr', 'desc', 'devtools',
  'dials', 'diffobj', 'digest', 'dplyr', 'DT', 'dtplyr', 'e1071', 'evaluate',
  'fansi', 'farver', 'forcats', 'foreach', 'forecast', 'fs', 'furrr', 'future',
  'future.apply', 'gargle', 'generics', 'gert', 'ggplot2', 'gh', 'git2r',
  'gitcreds', 'globals', 'glue', 'googleshee

In [5]:
BiocManager::install("biomaRt")

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.18), R 4.1.1 (2021-08-10)

Installing package(s) 'biomaRt'

also installing the dependencies ‘zlibbioc’, ‘GenomeInfoDbData’, ‘XVector’, ‘GenomeInfoDb’, ‘BiocGenerics’, ‘png’, ‘Biostrings’, ‘Biobase’, ‘IRanges’, ‘KEGGREST’, ‘filelock’, ‘XML’, ‘AnnotationDbi’, ‘BiocFileCache’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'backports', 'blob', 'brew', 'brio', 'broom', 'bslib', 'callr',
  'caret', 'class', 'cli', 'clipr', 'colorspace', 'commonmark', 'conflicted',
  'covr', 'cpp11', 'crayon', 'credentials', 'crosstalk', 'curl', 'data.table',
  'DBI', 'dbplyr', 'desc', 'devtools', 'dials', 'diffobj', 'digest', 'dplyr',
  'DT', 'dtplyr', 'e1071', 'evaluate', 'fansi', 'farver', 'forcats', 'foreach',
  'forecast', 'fs', 'furrr', 'future', 'f

In [6]:
library("devtools")

Loading required package: usethis



#### First time through - Received a Warning that rhdf5 not available for this version of R

Looked up our version and google searched R 4.1.1 rhdf5
    
Can install using `BiocManager::install("rhdf5")`

Pactherlab says to install `*rhdf5*` first

In [7]:
BiocManager::install("rhdf5", force=TRUE)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.18), R 4.1.1 (2021-08-10)

Installing package(s) 'rhdf5'

also installing the dependencies ‘Rhdf5lib’, ‘rhdf5filters’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'backports', 'blob', 'brew', 'brio', 'broom', 'bslib', 'callr',
  'caret', 'class', 'cli', 'clipr', 'colorspace', 'commonmark', 'conflicted',
  'covr', 'cpp11', 'crayon', 'credentials', 'crosstalk', 'curl', 'data.table',
  'DBI', 'dbplyr', 'desc', 'devtools', 'dials', 'diffobj', 'digest', 'dplyr',
  'DT', 'dtplyr', 'e1071', 'evaluate', 'fansi', 'farver', 'forcats', 'foreach',
  'forecast', 'fs', 'furrr', 'future', 'future.apply', 'gargle', 'generics',
  'gert', 'ggplot2', 'gh', 'git2r', 'gitcreds', 'globals', 'glue',
  'googlesheets4', 'gower', 'gtable', 'hardhat', 'have

In [8]:
library (rhdf5)


#### Issues

Noted in issues https://github.com/pachterlab/sleuth/issues/259 -- follow the instructions from [Paast](https://github.com/pachterlab/sleuth/issues/259#issuecomment-966270599)

Install rhdf5 as noted above.

Load the library

##### Clone sleuth and install after editing the file

Change directory to the top working directory in this etheral machine.

```bash
cd /sbgenomics/workspace
```

now clone the library 

```bash
git clone https://github.com/pachterlab/sleuth.git
```

edit NAMESPACE as the instructions note - to remove the dependency remove the last line to remove the reference to **rhdf5**

And then run the install.


In [9]:
devtools::install('../../sleuth/')

rlang        (0.4.11 -> 1.0.6 ) [CRAN]
fs           (1.5.0  -> 1.5.2 ) [CRAN]
digest       (0.6.27 -> 0.6.29) [CRAN]
memoise      (2.0.0  -> 2.0.1 ) [CRAN]
sass         (0.4.0  -> 0.4.2 ) [CRAN]
jsonlite     (1.7.2  -> 1.8.2 ) [CRAN]
htmltools    (0.5.2  -> 0.5.3 ) [CRAN]
magrittr     (2.0.1  -> 2.0.3 ) [CRAN]
later        (1.2.0  -> 1.3.0 ) [CRAN]
Rcpp         (1.0.7  -> 1.0.9 ) [CRAN]
stringi      (1.7.4  -> 1.7.8 ) [CRAN]
glue         (1.4.2  -> 1.6.2 ) [CRAN]
withr        (2.4.2  -> 2.5.0 ) [CRAN]
fansi        (0.5.0  -> 1.0.3 ) [CRAN]
cli          (3.0.1  -> 3.4.1 ) [CRAN]
pillar       (1.6.2  -> 1.8.1 ) [CRAN]
vctrs        (0.3.8  -> 0.4.2 ) [CRAN]
tidyselect   (1.1.1  -> 1.2.0 ) [CRAN]
tibble       (3.1.4  -> 3.1.8 ) [CRAN]
lifecycle    (1.0.0  -> 1.0.3 ) [CRAN]
generics     (0.1.0  -> 0.1.3 ) [CRAN]
colorspace   (2.0-2  -> 2.0-3 ) [CRAN]
viridisLite  (0.4.0  -> 0.4.1 ) [CRAN]
RColorBrewer (1.1-2  -> 1.1-3 ) [CRAN]
farver       (2.1.0  -> 2.1.1 ) [CRAN]
bslib        (0.3.0  -> 0

Skipping 1 packages not available: rhdf5

Installing 46 packages: rlang, fs, digest, memoise, sass, jsonlite, htmltools, magrittr, later, Rcpp, stringi, glue, withr, fansi, cli, pillar, vctrs, tidyselect, tibble, lifecycle, generics, colorspace, viridisLite, RColorBrewer, farver, bslib, commonmark, crayon, fontawesome, mime, httpuv, gtable, scales, stringr, plyr, cpp11, purrr, dplyr, isoband, aggregation, shiny, pheatmap, matrixStats, tidyr, data.table, ggplot2

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



✔  checking for file ‘/sbgenomics/workspace/sleuth/DESCRIPTION’
─  preparing ‘sleuth’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
   Omitted ‘LazyData’ from DESCRIPTION
─  building ‘sleuth_0.30.0.tar.gz’
   
Running /opt/conda/lib/R/bin/R CMD INSTALL \
  /tmp/RtmpHvladK/sleuth_0.30.0.tar.gz --install-tests 
* installing to library ‘/opt/conda/lib/R/library’
* installing *source* package ‘sleuth’ ...
** using staged installation
** R
** inst
** tests
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (sleuth)


In [10]:
library(sleuth)

We have successfully run Kallisto with Kallisto Quantitation.

Results may be found after running an application on Cavatica here:

```bash
/sbgenomics/project-files/
```

For this analysis we will use the results from the run using `metadata_ten_samples_only_txt`

Results are in:

```bash
/sbgenomics/project-files/ten_samples_expression_matrix.tpm.txt
```

### Parsing metadata

A sleuth analysis is dependent on a metadata file, which describes the experimental design, the sample names, conditions and covariates. The metadata file is external to sleuth, and must be prepared prior to analysis. A metadata file should have been downloaded along with the kallisto quantifications. The first step in a sleuth analysis is loading of the metadata file. You might need the path in read_table below to where you have downloaded the kallisto dataset, so that the path directs to the sample_table.txt. We then select the relevant columns of the metadata.

In our case, I used:

```bash
/sbgenomics/project-files/metadata_ten_samples_only.csv
```

In [11]:
metadata <- read.table('/sbgenomics/project-files/metadata_ten_samples_only.csv', sep=",", header=TRUE, stringsAsFactors = FALSE)

In [12]:
head(metadata, n=20)

,File.name,Case.ID,subject,Sample.ID,sample,Gender,Disease.type,Paired.end
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,PAXSBH-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r2.fq.gz,PAXSBH,PAXSBH,PAXSBH-03A-01R,PAXSBH-03A-01R,Female,TAM,1
2,PAXSBH-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r1.fq.gz,PAXSBH,PAXSBH,PAXSBH-03A-01R,PAXSBH-03A-01R,Female,TAM,2
3,PAXWGW-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r2.fq.gz,PAXWGW,PAXWGW,PAXWGW-03A-01R,PAXWGW-03A-01R,Female,TAM,1
4,PAXWGW-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r1.fq.gz,PAXWGW,PAXWGW,PAXWGW-03A-01R,PAXWGW-03A-01R,Female,TAM,2
5,PASNSP-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r1.fq.gz,PASNSP,PASNSP,PASNSP-03A-01R,PASNSP-03A-01R,Male,TAM,1
6,PASNSP-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r2.fq.gz,PASNSP,PASNSP,PASNSP-03A-01R,PASNSP-03A-01R,Male,TAM,2
7,PASWXF-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r1.fq.gz,PASWXF,PASWXF,PASWXF-03A-01R,PASWXF-03A-01R,Male,TAM,1
8,PASWXF-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r2.fq.gz,PASWXF,PASWXF,PASWXF-03A-01R,PASWXF-03A-01R,Male,TAM,2
9,PASXCL-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r2.fq.gz,PASXCL,PASXCL,PASXCL-03A-01R,PASXCL-03A-01R,Male,TAM,1


There is an error in the last sample detail - where the paired should read `2` and not NA.  So I copied the file to a local directory and corrected it -- it is corrected permanently now - but for this run through you can see:
```bash
cp /sbgenomics/project-files/metadata_ten_samples_only.csv /sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data
```

where I edited the file and now will read this one in.

In [47]:
metadata <- read.table('/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/metadata_ten_samples_only.csv', sep=",", header=TRUE, stringsAsFactors = FALSE)

In [48]:
head(metadata)

,oFile.name,Case.ID,subject,Sample.ID,sample,Gender,Disease.type,Paired.end,Abundance
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,PAXSBH-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r1.fq.gz,PAXSBH,PAXSBH,PAXSBH-03A-01R,PAXSBH-03A-01R,Female,TAM,1,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAXSBH-03A-01R.kallisto_quant.abundance.tsv
2,PAXSBH-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r1.fq.gz,PAXSBH,PAXSBH,PAXSBH-03A-01R,PAXSBH-03A-01R,Female,TAM,2,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAXSBH-03A-01R.kallisto_quant.abundance.tsv
3,PAXWGW-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r2.fq.gz,PAXWGW,PAXWGW,PAXWGW-03A-01R,PAXWGW-03A-01R,Female,TAM,1,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAXWGW-03A-01R.kallisto_quant.abundance.tsv
4,PAXWGW-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r1.fq.gz,PAXWGW,PAXWGW,PAXWGW-03A-01R,PAXWGW-03A-01R,Female,TAM,2,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAXWGW-03A-01R.kallisto_quant.abundance.tsv
5,PASNSP-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r1.fq.gz,PASNSP,PASNSP,PASNSP-03A-01R,PASNSP-03A-01R,Male,TAM,1,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PASNSP-03A-01R.kallisto_quant.abundance.tsv
6,PASNSP-03A-01R_RBS_withJunctionsOnGenome_dupsFlagged_r2.fq.gz,PASNSP,PASNSP,PASNSP-03A-01R,PASNSP-03A-01R,Male,TAM,2,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PASNSP-03A-01R.kallisto_quant.abundance.tsv


In [49]:
dim(metadata)

[1] 20  9

In [50]:
metadata <- dplyr::select(metadata, c('Case.ID', 'Sample.ID', 'Gender', 'Disease.type', 'Abundance'))

In [51]:
head(metadata)

,Case.ID,Sample.ID,Gender,Disease.type,Abundance
,<chr>,<chr>,<chr>,<chr>,<chr>
1,PAXSBH,PAXSBH-03A-01R,Female,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAXSBH-03A-01R.kallisto_quant.abundance.tsv
2,PAXSBH,PAXSBH-03A-01R,Female,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAXSBH-03A-01R.kallisto_quant.abundance.tsv
3,PAXWGW,PAXWGW-03A-01R,Female,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAXWGW-03A-01R.kallisto_quant.abundance.tsv
4,PAXWGW,PAXWGW-03A-01R,Female,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAXWGW-03A-01R.kallisto_quant.abundance.tsv
5,PASNSP,PASNSP-03A-01R,Male,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PASNSP-03A-01R.kallisto_quant.abundance.tsv
6,PASNSP,PASNSP-03A-01R,Male,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PASNSP-03A-01R.kallisto_quant.abundance.tsv


In [52]:
metadata <- dplyr::distinct(metadata)

In [53]:
head(metadata)

,Case.ID,Sample.ID,Gender,Disease.type,Abundance
,<chr>,<chr>,<chr>,<chr>,<chr>
1,PAXSBH,PAXSBH-03A-01R,Female,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAXSBH-03A-01R.kallisto_quant.abundance.tsv
2,PAXWGW,PAXWGW-03A-01R,Female,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAXWGW-03A-01R.kallisto_quant.abundance.tsv
3,PASNSP,PASNSP-03A-01R,Male,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PASNSP-03A-01R.kallisto_quant.abundance.tsv
4,PASWXF,PASWXF-03A-01R,Male,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PASWXF-03A-01R.kallisto_quant.abundance.tsv
5,PASXCL,PASXCL-03A-01R,Male,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PASXCL-03A-01R.kallisto_quant.abundance.tsv
6,PAVZTK,PAVZTK-09A-01R,Female,DS-AML,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAVZTK-09A-01R.kallisto_quant.abundance.tsv


In [54]:
dim(metadata)

[1] 10  5

In [55]:
metadata <- dplyr::rename(metadata, sample = Sample.ID)

Need to rename a colump as well to `path` where we have `Abundance`

In [56]:
metadata <- dplyr::rename(metadata, path = Abundance)

In [57]:
head(metadata)

,Case.ID,sample,Gender,Disease.type,path
,<chr>,<chr>,<chr>,<chr>,<chr>
1,PAXSBH,PAXSBH-03A-01R,Female,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAXSBH-03A-01R.kallisto_quant.abundance.tsv
2,PAXWGW,PAXWGW-03A-01R,Female,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAXWGW-03A-01R.kallisto_quant.abundance.tsv
3,PASNSP,PASNSP-03A-01R,Male,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PASNSP-03A-01R.kallisto_quant.abundance.tsv
4,PASWXF,PASWXF-03A-01R,Male,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PASWXF-03A-01R.kallisto_quant.abundance.tsv
5,PASXCL,PASXCL-03A-01R,Male,TAM,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PASXCL-03A-01R.kallisto_quant.abundance.tsv
6,PAVZTK,PAVZTK-09A-01R,Female,DS-AML,/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/PAVZTK-09A-01R.kallisto_quant.abundance.tsv


#### biomaRt - how to use

Following instructions from the [ensembl site](https://grch37.ensembl.org/info/data/biomart/biomart_r_package.html)

In [24]:
library(biomaRt)

In [25]:
listEnsembl()

biomart,version
<chr>,<chr>
genes,Ensembl Genes 107
mouse_strains,Mouse strains 107
snps,Ensembl Variation 107
regulation,Ensembl Regulation 107


In [26]:
mart <- biomaRt::useMart(biomart="ensembl", 
                     dataset = "hsapiens_gene_ensembl",
                        host = "https://useast.ensembl.org")

In [68]:
ttg <- biomaRt::getBM(
  attributes = c("ensembl_transcript_id", "transcript_version",
  "ensembl_gene_id", "external_gene_name", "description",
  "transcript_biotype"),
  mart = mart)


In [69]:
ttg <- dplyr::rename(ttg, target_id = ensembl_transcript_id,
  ens_gene = ensembl_gene_id, ext_gene = external_gene_name)


In [70]:
ttg <- dplyr::select(ttg, c('target_id', 'ens_gene', 'ext_gene'))
head(ttg)

,target_id,ens_gene,ext_gene
,<chr>,<chr>,<chr>
1,ENST00000387314,ENSG00000210049,MT-TF
2,ENST00000389680,ENSG00000211459,MT-RNR1
3,ENST00000387342,ENSG00000210077,MT-TV
4,ENST00000387347,ENSG00000210082,MT-RNR2
5,ENST00000386347,ENSG00000209082,MT-TL1
6,ENST00000361390,ENSG00000198888,MT-ND1


In [44]:
ttg <- read.table('/sbgenomics/workspace/pediatric-DS-AML-TAM-Analysis/data/ttg.csv', sep=",", header=TRUE, stringsAsFactors = FALSE)

In [45]:
head(ttg)

,target_id,ens_gene,ext_gene
,<chr>,<chr>,<chr>
1,ENST00000387314,ENSG00000210049,MT-TF
2,ENST00000389680,ENSG00000211459,MT-RNR1
3,ENST00000387342,ENSG00000210077,MT-TV
4,ENST00000387347,ENSG00000210082,MT-RNR2
5,ENST00000386347,ENSG00000209082,MT-TL1
6,ENST00000361390,ENSG00000198888,MT-ND1


The resulting table contains Ensembl gene names (‘ens_gene’) and the associated transcripts (‘target_id’). Note that the gene-transcript mapping must be compatible with the transcriptome used with kallisto. In other words, to use Ensembl transcript-gene associations kallisto was run using the Ensembl transcriptome.

#### Preparing the analysis

The next step is to build a sleuth object. The sleuth object contains specification of the experimental design, a map describing grouping of transcripts into genes (or other groups), and a number of user specific parameters. In the example that follows, metadata is the experimental design and target_mapping describes the transcript groupings into genes previously constructed. Furthermore, we provide an aggregation_column, the column name of in ‘target_mapping’ table that is used to aggregate the transcripts. When both ‘target_mapping’ and ‘aggregation_column’ are provided, sleuth will automatically run in gene mode, returning gene differential expression results that came from the aggregation of transcript p-values.


In [71]:
so <- sleuth_prep(metadata, 
                  target_mapping = ttg, 
                  aggregation_column = 'ens_gene',
                  gene_mode = TRUE,
                  extra_bootstrap_summary = TRUE)

'gene_mode' is TRUE. Sleuth will do counts aggregation at the gene level for downstream normalization, transformation, and modeling steps, as well as for plotting and results.

reading in kallisto results

dropping unused factor levels

.
.
.
.
.
.
.
.
.
.




ERROR: Error in if (any(counts_test$total == 0)) {: missing value where TRUE/FALSE needed


In [72]:
sleuth_prep


function (sample_to_covariates, full_model = NULL, target_mapping = NULL, 
    aggregation_column = NULL, num_cores = max(1L, parallel::detectCores() - 
        1L), ...) 
{
    extra_opts <- list(...)
    if ("extra_bootstrap_summary" %in% names(extra_opts)) {
        extra_bootstrap_summary <- extra_opts$extra_bootstrap_summary
    }
    else {
        extra_bootstrap_summary <- FALSE
    }
    if ("read_bootstrap_tpm" %in% names(extra_opts)) {
        read_bootstrap_tpm <- extra_opts$read_bootstrap_tpm
    }
    else {
        read_bootstrap_tpm <- FALSE
    }
    if ("max_bootstrap" %in% names(extra_opts)) {
        max_bootstrap <- extra_opts$max_bootstrap
    }
    else {
        max_bootstrap <- NULL
    }
    if ("filter_fun" %in% names(extra_opts)) {
        filter_fun <- extra_opts$filter_fun
    }
    else {
        filter_fun <- basic_filter
    }
    if ("filter_target_id" %in% names(extra_opts)) {
        filter_target_id <- extra_opts$filter_target_id
    }
    else {
        filter_target_id <- NULL
    }
    if ("normalize" %in% names(extra_opts)) {
        normalize <- extra_opts$normalize
    }
    else {
        normalize <- TRUE
    }
    if ("norm_fun_counts" %in% names(extra_opts)) {
        norm_fun_counts <- extra_opts$norm_fun_counts
    }
    else {
        norm_fun_counts <- norm_factors
    }
    if ("norm_fun_tpm" %in% names(extra_opts)) {
        norm_fun_tpm <- extra_opts$norm_fun_tpm
    }
    else {
        norm_fun_tpm <- norm_factors
    }
    if ("transform_fun_counts" %in% names(extra_opts)) {
        transform_fun_counts <- extra_opts$transform_fun_counts
    }
    else {
        transform_fun_counts <- log_transform
    }
    if ("transform_fun_tpm" %in% names(extra_opts)) {
        transform_fun_tpm <- extra_opts$transform_fun_tpm
    }
    else {
        transform_fun_tpm <- identity
    }
    if ("gene_mode" %in% names(extra_opts)) {
        gene_mode <- extra_opts$gene_mode
    }
    else {
        gene_mode <- FALSE
    }
    if (!is(sample_to_covariates, "data.frame")) {
        stop(paste0("'", substitute(sample_to_covariates), "' (sample_to_covariates) must be a data.frame"))
    }
    if (!is(full_model, "formula") && !is(full_model, "matrix") && 
        !is.null(full_model)) {
        stop(paste0("'", substitute(full_model), "' (full_model) must be a formula or a matrix"))
    }
    if (!("sample" %in% colnames(sample_to_covariates))) {
        stop(paste0("'", substitute(sample_to_covariates), "' (sample_to_covariates) must contain a column named 'sample'"))
    }
    if (!("path" %in% colnames(sample_to_covariates))) {
        stop(paste0("'", substitute(sample_to_covariates)), "' (sample_to_covariates) must contain a column named 'path'")
    }
    if (!is.null(target_mapping) && !is(target_mapping, "data.frame")) {
        stop(paste0("'", substitute(target_mapping), "' (target_mapping) must be a data.frame or NULL"))
    }
    else if (is(target_mapping, "data.frame")) {
        if (!("target_id" %in% colnames(target_mapping))) {
            stop(paste0("'", substitute(target_mapping), "' (target_mapping) must contain a column named 'target_id'"))
        }
    }
    if (!is.null(max_bootstrap) && max_bootstrap <= 0) {
        stop("max_bootstrap must be > 0")
    }
    if (any(is.na(sample_to_covariates))) {
        warning("Your 'sample_to_covariates' data.frame contains NA values. This will likely cause issues later.")
    }
    if (is(full_model, "matrix") && nrow(full_model) != nrow(sample_to_covariates)) {
        stop("The design matrix number of rows are not equal to the number of rows in the sample_to_covariates argument.")
    }
    if (!is(filter_fun, "function")) {
        stop("filter_fun must be a function")
    }
    if (!is.null(filter_target_id) & !is.character(filter_target_id)) {
        stop("if filter_target_id is used, it must be a character vector")
    }
    if (!is(norm_fun_counts, "function")) {
        stop("norm_fun_counts must be a function")


In [62]:
ttg.df <- data.frame (ttg)

[1] 0